In [2]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scFunctions as sf
import pickle

import warnings
warnings.filterwarnings('ignore')

sc.settings.verbosity = 3


%config Completer.use_jedi = False

In [3]:
with open("../data/adata_new_orig.pickle", "rb") as f:
    adata = pickle.load(f)

In [4]:
df = pd.read_csv("../data/xenium_breast_genes.csv", index_col = 0)

In [5]:
xenium_panel = df.index.values
xenium_panel

array(['ABCC11', 'ACTA2', 'ACTG2', 'ADAM9', 'ADGRE5', 'ADH1B', 'ADIPOQ',
       'AGR3', 'AIF1', 'AKR1C1', 'AKR1C3', 'ALDH1A3', 'ANGPT2', 'ANKRD28',
       'ANKRD29', 'ANKRD30A', 'APOBEC3A', 'APOBEC3B', 'APOC1', 'AQP1',
       'AQP3', 'AR', 'AVPR1A', 'BACE2', 'BANK1', 'BASP1', 'C15orf48',
       'C1QA', 'C1QC', 'C2orf42', 'C5orf46', 'C6orf132', 'CAV1', 'CAVIN2',
       'CCDC6', 'CCDC80', 'CCL5', 'CCL8', 'CCND1', 'CCPG1', 'CCR7',
       'CD14', 'CD163', 'CD19', 'CD247', 'CD27', 'CD274', 'CD3E', 'CD3G',
       'CD4', 'CD68', 'CD69', 'CD79A', 'CD79B', 'CD80', 'CD83', 'CD86',
       'CD8A', 'CD9', 'CD93', 'CDC42EP1', 'CDH1', 'CEACAM6', 'CEACAM8',
       'CENPF', 'CLEC14A', 'CLEC9A', 'CLECL1', 'CLIC6', 'CPA3',
       'CRISPLD2', 'CTH', 'CTLA4', 'CTSG', 'CTTN', 'CX3CR1', 'CXCL12',
       'CXCL16', 'CXCL5', 'CXCR4', 'CYTIP', 'DAPK3', 'DMKN', 'DNAAF1',
       'DNTTIP1', 'DPT', 'DSC2', 'DSP', 'DST', 'DUSP2', 'DUSP5', 'EGFL7',
       'EGFR', 'EIF4EBP1', 'ENAH', 'EPCAM', 'ERBB2', 'ERN1', 'ESM1',
 

## Converting human to mouse genes

### Mousipy

In [32]:
from mousipy import translate

test = translate(adata.raw.copy())

100%|████████████████████████████████████| 25572/25572 [00:53<00:00, 477.00it/s]


Found direct orthologs for 15955 genes.
Found multiple orthologs for 459 genes.
Found no orthologs for 8175 genes.
Found no index in biomart for 983 genes.


AttributeError: can't set attribute

In [8]:
adata

AnnData object with n_obs × n_vars = 119630 × 2922
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Experiment_ID', 'Stage', 'New_Cell_ID', 'Population', 'Mouse_ID', 'Week_post Cre_activation', 'Sub_class_Stage', 'Mammary_glands', 'mouse_age_(Week)', 'new_orig'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'log1p', 'hvg', 'pca', 'Stage_colors', 'neighbors', 'umap', 'new_orig_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [26]:
test

AnnData object with n_obs × n_vars = 119630 × 2655
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Experiment_ID', 'Stage', 'New_Cell_ID', 'Population', 'Mouse_ID', 'Week_post Cre_activation', 'Sub_class_Stage', 'Mammary_glands', 'mouse_age_(Week)', 'new_orig'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'original_gene_symbol'
    uns: 'log1p', 'hvg', 'pca', 'Stage_colors', 'neighbors', 'umap', 'new_orig_colors'
    obsm: 'X_pca', 'X_umap'

In [14]:
liste_keep = []
for i in xenium_panel:
    if i in test.var_names:
        liste_keep.append(i)
len(liste_keep)

161

In [27]:
adata_test = test.copy()

In [28]:
test.raw = test

In [29]:
adata_test = adata_test[:, liste_keep]

In [31]:
sf.pp_PCA(adata_test)

normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes


ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan]).
You can drop duplicate edges by setting the 'duplicates' kwarg

### Biomart

In [15]:
from gseapy import Biomart
bm = Biomart()

h2m = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'mmusculus_homolog_ensembl_gene',
                           'mmusculus_homolog_associated_gene_name'])

In [19]:
h2m[h2m["mmusculus_homolog_ensembl_gene"].isna() == False]

,ensembl_gene_id,external_gene_name,mmusculus_homolog_ensembl_gene,mmusculus_homolog_associated_gene_name
5,ENSG00000198888,MT-ND1,ENSMUSG00000064341,mt-Nd1
9,ENSG00000198763,MT-ND2,ENSMUSG00000064345,mt-Nd2
15,ENSG00000198804,MT-CO1,ENSMUSG00000064351,mt-Co1
18,ENSG00000198712,MT-CO2,ENSMUSG00000064354,mt-Co2
20,ENSG00000228253,MT-ATP8,ENSMUSG00000064356,mt-Atp8
...,...,...,...,...
77788,ENSG00000175087,PDIK1L,ENSMUSG00000050890,Pdik1l
77789,ENSG00000197245,FAM110D,ENSMUSG00000050105,Fam110d
77790,ENSG00000282872,C1orf232,ENSMUSG00000108398,Gm30191
77792,ENSG00000142684,ZNF593,ENSMUSG00000028840,Zfp593


In [20]:
h2m

,ensembl_gene_id,external_gene_name,mmusculus_homolog_ensembl_gene,mmusculus_homolog_associated_gene_name
0,ENSG00000210049,MT-TF,NaN,NaN
1,ENSG00000211459,MT-RNR1,NaN,NaN
2,ENSG00000210077,MT-TV,NaN,NaN
3,ENSG00000210082,MT-RNR2,NaN,NaN
4,ENSG00000209082,MT-TL1,NaN,NaN
...,...,...,...,...
77797,ENSG00000288629,NaN,NaN,NaN
77798,ENSG00000288678,NaN,NaN,NaN
77799,ENSG00000290825,DDX11L2,NaN,NaN
77800,ENSG00000227232,WASH7P,NaN,NaN


In [51]:
dico = {}
for i in xenium_panel:
    dico[i] = []
    dico[i] = list(h2m[h2m["external_gene_name"] == i].mmusculus_homolog_associated_gene_name.values)

In [52]:
dico

{'ABCC11': [nan],
 'ACTA2': ['Acta2'],
 'ACTG2': ['Actg2'],
 'ADAM9': [nan, 'Adam9'],
 'ADGRE5': ['Adgre5'],
 'ADH1B': ['Adh1'],
 'ADIPOQ': ['Adipoq'],
 'AGR3': ['Agr3'],
 'AIF1': [nan, nan, nan, nan, nan, 'Aif1'],
 'AKR1C1': [nan],
 'AKR1C3': [nan],
 'ALDH1A3': ['Aldh1a3'],
 'ANGPT2': ['Angpt2'],
 'ANKRD28': ['Ankrd28'],
 'ANKRD29': ['Ankrd29'],
 'ANKRD30A': [nan],
 'APOBEC3A': [nan, nan],
 'APOBEC3B': [nan],
 'APOC1': ['Apoc1'],
 'AQP1': ['Aqp1'],
 'AQP3': ['Aqp3'],
 'AR': ['Ar'],
 'AVPR1A': ['Avpr1a'],
 'BACE2': ['Bace2'],
 'BANK1': ['Bank1'],
 'BASP1': ['Basp1'],
 'C15orf48': ['AA467197'],
 'C1QA': ['C1qa'],
 'C1QC': ['C1qc'],
 'C2orf42': ['C87436'],
 'C5orf46': ['Gm94'],
 'C6orf132': ['AI661453'],
 'CAV1': ['Cav1'],
 'CAVIN2': ['Cavin2'],
 'CCDC6': ['Ccdc6'],
 'CCDC80': ['Ccdc80'],
 'CCL5': [nan, 'Ccl5'],
 'CCL8': ['Ccl8', 'Ccl12'],
 'CCND1': ['Ccnd1'],
 'CCPG1': ['Ccpg1'],
 'CCR7': ['Ccr7'],
 'CD14': ['Cd14'],
 'CD163': ['Cd163'],
 'CD19': ['Cd19'],
 'CD247': ['Cd247'],
 'CD27': 

In [76]:
xenium_mouse = []
for i in dico.values():
    if len(i) > 0:
        for j in i:
            if pd.notna(j):
                xenium_mouse.append(j)

In [83]:
adata_genes = adata.var_names.values

In [90]:
j = 0
for i in xenium_mouse:
    if i not in adata_genes:
        j+=1
print(j)
print(len(xenium_mouse))

138
312


In [93]:
adata_genes.sort()
for i in adata_genes:
    print(i)

0610040F04Rik
0610043K17Rik
1200007C13Rik
1500015L24Rik
1600002D24Rik
1600010M07Rik
1600020E01Rik
1700001K19Rik
1700016F12Rik
1700016P03Rik
1700054A03Rik
1700126G02Rik
1810006J02Rik
2010013B24Rik
2200002D01Rik
2310001H17Rik
2610035D17Rik
2610307P16Rik
2700054A10Rik
2810013P06Rik
2810403D21Rik
2900060B14Rik
4732465J04Rik
4833407H14Rik
4930404I05Rik
4930417O13Rik
4930430E12Rik
4930469K13Rik
4930549G23Rik
4933406I18Rik
4933406J09Rik
4933432I03Rik
5730522E02Rik
5830416I19Rik
5830428M24Rik
6030443J06Rik
8430422H06Rik
9130230L23Rik
9530026P05Rik
9830107B12Rik
A230059L01Rik
A330023F24Rik
A330040F15Rik
A430093F15Rik
A4galt
A530013C23Rik
A530064D06Rik
A630001O12Rik
A630023P12Rik
A730081D07Rik
A930007I19Rik
A930015D03Rik
AA467197
AB124611
AC125149.3
AC168977.1
AI838599
AI839979
AU019990
AW011738
AW112010
AW554918
AY036118
Aatk
Abca7
Abca8a
Abca8b
Abcb1a
Abcb1b
Abcc9
Abcg2
Abcg3
Abhd17b
Abi3bp
Ablim1
Ablim3
Abr
Abtb2
Acacb
Ace
Acer2
Acer3
Ackr1
Ackr3
Acod1
Acox1
Acoxl
Acp5
Acsl1
Acta1
Acta2
Actg2